In [11]:
import os,sys
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import glob as glob
import numpy as np
import warnings
from scipy.stats import ttest_ind,f_oneway

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from plot_functions import plot_count, plot_box, plot_box_category, plot_regplot, barplot_count_multicat, \
barplot_pct_multicat, plot_confusion_matrix
from nominal import *
from sklearn import preprocessing


%reload_ext autoreload
%autoreload 2
%matplotlib inline
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

warnings.filterwarnings(action='once')
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
data_path = os.path.join(os.getcwd(),"Animal State Prediction - dataset")
os.chdir(data_path)
print(data_path)

E:\Projects\ZS_Hackerrank_Feb2020\Animal State Prediction - dataset


In [3]:
train_df = pd.read_csv("train.csv")
train_df.head()
# train_df["intake_datetime"] = pd.to_datetime("intake_datetime")
# train_df["date_of_birth"] = pd.to_datetime("date_of_birth")
# train_df["outcome_datetime"] = pd.to_datetime("outcome_datetime")

,animal_id_outcome,dob_year,dob_month,age_upon_intake,animal_type,breed,color,intake_condition,intake_type,sex_upon_intake,count,age_upon_intake_(days),age_upon_intake_(years),age_upon_intake_age_group,intake_datetime,intake_month,intake_year,intake_monthyear,intake_weekday,intake_hour,intake_number,time_in_shelter,time_in_shelter_days,age_upon_outcome,date_of_birth,sex_upon_outcome,age_upon_outcome_(days),age_upon_outcome_(years),age_upon_outcome_age_group,outcome_datetime,outcome_month,outcome_year,outcome_monthyear,outcome_weekday,outcome_hour,outcome_number,outcome_type
0,A006100,2007,7,10 years,Dog,Spinone Italiano Mix,Yellow/White,Normal,Stray,Neutered Male,1,3650,10.0,"(7.5, 10.0]",2017-12-07 00:00:00,12,2017,2017-12,Thursday,14,1.0,0 days 14:07:00.000000000,0.588194,10 years,2007-07-09 00:00:00,Neutered Male,3650,10.0,"(7.5, 10.0]",2017-12-07 14:07:00,12,2017,2017-12,Thursday,0,1.0,Return to Owner
1,A006100,2007,7,7 years,Dog,Spinone Italiano Mix,Yellow/White,Normal,Public Assist,Neutered Male,1,2555,7.0,"(5.0, 7.5]",2014-12-19 10:21:00,12,2014,2014-12,Friday,10,2.0,1 days 06:14:00.000000000,1.259722,7 years,2007-07-09 00:00:00,Neutered Male,2555,7.0,"(5.0, 7.5]",2014-12-20 16:35:00,12,2014,2014-12,Saturday,16,2.0,Return to Owner
2,A006100,2007,7,6 years,Dog,Spinone Italiano Mix,Yellow/White,Normal,Public Assist,Neutered Male,1,2190,6.0,"(5.0, 7.5]",2014-03-07 14:26:00,3,2014,2014-03,Friday,14,3.0,1 days 02:44:00.000000000,1.113889,6 years,2007-07-09 00:00:00,Neutered Male,2190,6.0,"(5.0, 7.5]",NaN,3,2014,2014-03,Saturday,17,3.0,Return to Owner
3,A047759,2004,4,10 years,Dog,Dachshund,Tricolor,Normal,Owner Surrender,Neutered Male,1,3650,10.0,"(7.5, 10.0]",2014-04-02 15:55:00,4,2014,2014-04,Wednesday,15,1.0,4 days 23:17:00.000000000,4.970139,10 years,2004-04-02 00:00:00,Neutered Male,3650,10.0,"(7.5, 10.0]",2014-04-07 15:12:00,4,2014,2014-04,Monday,15,1.0,Missing
4,A134067,1997,10,16 years,Dog,Shetland Sheepdog,Brown/White,Injured,Public Assist,Neutered Male,1,5840,16.0,"(15.0, 17.5]",2013-11-16 09:02:00,11,2013,2013-11,Saturday,9,1.0,0 days 02:52:00.000000000,0.119444,16 years,1997-10-16 00:00:00,Neutered Male,5840,16.0,"(15.0, 17.5]",2013-11-16 11:54:00,11,2013,2013-11,Saturday,11,1.0,Return to Owner


In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47803 entries, 0 to 47802
Data columns (total 37 columns):
animal_id_outcome             47803 non-null object
dob_year                      47803 non-null int64
dob_month                     47803 non-null int64
age_upon_intake               47803 non-null object
animal_type                   47803 non-null object
breed                         47803 non-null object
color                         47803 non-null object
intake_condition              47803 non-null object
intake_type                   47803 non-null object
sex_upon_intake               47802 non-null object
count                         47803 non-null int64
age_upon_intake_(days)        47803 non-null int64
age_upon_intake_(years)       47803 non-null float64
age_upon_intake_age_group     47803 non-null object
intake_datetime               47803 non-null object
intake_month                  47803 non-null int64
intake_year                   47803 non-null int64
intake_month

In [5]:
train_df["breed_simple"] = train_df.breed.apply(lambda x: x.split("/")[0])
train_df["breed_simple"] = train_df.breed_simple.apply(lambda x: x.replace(" Mix",""))
train_df["color_simple"] = train_df.color.apply(lambda x: x.split("/")[0])
#train_df.drop(["breed","color"],inplace=True,axis=1)

In [6]:
# COLS_DROP = ["dob_year","dob_month","age_upon_intake","age_upon_intake_(years)","age_upon_intake_age_group",\
#              "intake_datetime","intake_month","intake_year","intake_monthyear","intake_weekday","intake_hour","time_in_shelter",\
#              "age_upon_outcome","date_of_birth","age_upon_outcome_(years)","age_upon_outcome_age_group","outcome_datetime",\
#              "outcome_month","outcome_year","outcome_monthyear","outcome_weekday","outcome_hour","outcome_number"
#             ]

COLS_DROP = ["outcome_datetime"]
train_df.drop(COLS_DROP,axis=1,inplace=True)
train_df.dropna(how="any",inplace=True)
train_df.shape

(47802, 38)

## Feature importance

In [6]:

CAT_COLS = [
    "age_upon_intake","animal_type","intake_condition","intake_type","sex_upon_intake","age_upon_intake_age_group", \
    "intake_monthyear","intake_weekday","age_upon_outcome","sex_upon_outcome","outcome_monthyear","outcome_weekday", \
    "breed_simple","color_simple"
]

for col in CAT_COLS:
    print(f"{col} --> {theils_u(train_df[col],train_df['outcome_type'])}")

age_upon_intake --> 0.04075590282525965
animal_type --> 0.181789051754996
intake_condition --> 0.11063012702461726
intake_type --> 0.1792520123109551
sex_upon_intake --> 0.09429610593288003
age_upon_intake_age_group --> 0.05334850477134934
intake_monthyear --> 0.00932297932471609
intake_weekday --> 0.000917036901365201
age_upon_outcome --> 0.053788681614858565
sex_upon_outcome --> 0.13865449455385265
outcome_monthyear --> 0.009793694115191006
outcome_weekday --> 0.005990657094530605
breed_simple --> 0.06345029169616354
color_simple --> 0.026411652427103183


In [7]:
NUM_COLS = [
    "dob_year","dob_month","count","age_upon_intake_(days)","age_upon_intake_(years)","intake_month","intake_year",\
    "intake_hour","intake_number","time_in_shelter_days","age_upon_outcome_(days)","age_upon_outcome_(years)", \
            "outcome_month","outcome_year","outcome_hour","outcome_number"
]

for col in NUM_COLS:
    print(f"{col} --> {correlation_ratio(train_df['outcome_type'],train_df[col])}")

dob_year --> 0.31537449102942816
dob_month --> 0.022665507325972535
count --> 0.0
age_upon_intake_(days) --> 0.3321009990721741
age_upon_intake_(years) --> 0.3321009990721741
intake_month --> 0.0439706809957221
intake_year --> 0.13764688376038314
intake_hour --> 0.04218712428773912
intake_number --> 0.2292030193425763
time_in_shelter_days --> 0.2291022423424975
age_upon_outcome_(days) --> 0.3283702187812068
age_upon_outcome_(years) --> 0.32837021878120676
outcome_month --> 0.04432196388328209
outcome_year --> 0.13309341383596038
outcome_hour --> 0.2551599180069674
outcome_number --> 0.2292030193425763


## Modelling

In [9]:
train_df.columns

Index(['animal_id_outcome', 'dob_year', 'dob_month', 'age_upon_intake',
       'animal_type', 'breed', 'color', 'intake_condition', 'intake_type',
       'sex_upon_intake', 'count', 'age_upon_intake_(days)',
       'age_upon_intake_(years)', 'age_upon_intake_age_group',
       'intake_datetime', 'intake_month', 'intake_year', 'intake_monthyear',
       'intake_weekday', 'intake_hour', 'intake_number', 'time_in_shelter',
       'time_in_shelter_days', 'age_upon_outcome', 'date_of_birth',
       'sex_upon_outcome', 'age_upon_outcome_(days)',
       'age_upon_outcome_(years)', 'age_upon_outcome_age_group',
       'outcome_month', 'outcome_year', 'outcome_monthyear', 'outcome_weekday',
       'outcome_hour', 'outcome_number', 'outcome_type', 'breed_simple',
       'color_simple'],
      dtype='object')

In [7]:
NUM_COLS = [
    "dob_year","dob_month","count","age_upon_intake_(days)","age_upon_intake_(years)","intake_month","intake_year",\
    "intake_hour","intake_number","time_in_shelter_days","age_upon_outcome_(days)","age_upon_outcome_(years)", \
            "outcome_month","outcome_year","outcome_hour","outcome_number"
]

CAT_COLS = [
    "age_upon_intake","animal_type","intake_condition","intake_type","sex_upon_intake","age_upon_intake_age_group", \
    "intake_monthyear","intake_weekday","age_upon_outcome","sex_upon_outcome","outcome_monthyear","outcome_weekday", \
    "breed_simple","color_simple"
]

# NUM_COLS = [
#     'count','age_upon_intake_(days)', 'intake_number', 'time_in_shelter_days','age_upon_outcome_(days)'
# ]

# CAT_COLS = [
#     'animal_type','intake_condition', 'intake_type', 'sex_upon_intake','sex_upon_outcome'
# ]

RESPONSE = ["outcome_type"]

numerical_df = train_df[NUM_COLS]
categorical_df = pd.get_dummies(train_df[CAT_COLS])
response_df = train_df[RESPONSE]

for col in categorical_df.columns:
    if "]" in str(col) or "[" in str(col):
        new_name = str(col).replace("]",")").replace("[","(")
        categorical_df.rename({str(col):new_name},inplace=True,axis=1)
        print(f"Renamed from {col} ---> {new_name}")
        
all_df = pd.concat([numerical_df,categorical_df,response_df],axis=1)
print(f"Number of OHE features added: {all_df.shape[1]-len(NUM_COLS)-len(CAT_COLS)-1}")

Renamed from age_upon_intake_age_group_(-0.025, 2.5] ---> age_upon_intake_age_group_(-0.025, 2.5)
Renamed from age_upon_intake_age_group_(10.0, 12.5] ---> age_upon_intake_age_group_(10.0, 12.5)
Renamed from age_upon_intake_age_group_(12.5, 15.0] ---> age_upon_intake_age_group_(12.5, 15.0)
Renamed from age_upon_intake_age_group_(15.0, 17.5] ---> age_upon_intake_age_group_(15.0, 17.5)
Renamed from age_upon_intake_age_group_(17.5, 20.0] ---> age_upon_intake_age_group_(17.5, 20.0)
Renamed from age_upon_intake_age_group_(2.5, 5.0] ---> age_upon_intake_age_group_(2.5, 5.0)
Renamed from age_upon_intake_age_group_(20.0, 22.5] ---> age_upon_intake_age_group_(20.0, 22.5)
Renamed from age_upon_intake_age_group_(5.0, 7.5] ---> age_upon_intake_age_group_(5.0, 7.5)
Renamed from age_upon_intake_age_group_(7.5, 10.0] ---> age_upon_intake_age_group_(7.5, 10.0)
Number of OHE features added: 599


In [8]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn.metrics import classification_report 

In [12]:
#divide into train and test set
print("Training data--Test data Date split")
print(all_df.shape)
# Create X, y datasets and train,Val datasets
X = all_df[all_df.columns[:-1]]
#X = XGB_Train['Right1_min']
y = all_df[RESPONSE]

X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.25,random_state=0)
# X_train = np.array(X_train).reshape(-1,1)
# X_val = np.array(X_val).reshape(-1,1)
# y_train = np.array(y_train).reshape(-1,1)
# y_val = np.array(y_val).reshape(-1,1)
#, X_val, y_train, y_val
features = X_train.columns
print("Training data--Validation data X-y split")
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)
print(features)
#print("training data NG sample percentage",100*y_train.value_counts()[1]/sum(y_train.value_counts()))
#print("testing data NG sample percentage",100*y_val.value_counts()[1]/sum(y_val.value_counts()))

#Fitting XGB Classifier 
model=xgb.XGBClassifier(colsample_bytree=0.4,
                gamma=0,
                learning_rate=0.07,
                max_depth=3,
                min_child_weight=1.5,
                n_estimators=1000,
                reg_alpha=0.75,
                reg_lambda=0.45,
                subsample=0.6,
                seed=42)
model.fit(X_train,y_train)
y_pred = model.predict(data=X_val)
print (model)

Training data--Test data Date split
(47802, 630)
Training data--Validation data X-y split
(35851, 629) (11951, 629) (35851, 1) (11951, 1)
Index(['dob_year', 'dob_month', 'count', 'age_upon_intake_(days)',
       'age_upon_intake_(years)', 'intake_month', 'intake_year', 'intake_hour',
       'intake_number', 'time_in_shelter_days',
       ...
       'color_simple_Silver Lynx Point', 'color_simple_Silver Tabby',
       'color_simple_Tan', 'color_simple_Torbie', 'color_simple_Tortie',
       'color_simple_Tortie Point', 'color_simple_Tricolor',
       'color_simple_White', 'color_simple_Yellow',
       'color_simple_Yellow Brindle'],
      dtype='object', length=629)


C:\Users\Administrator\Anaconda3\lib\site-packages\xgboost\sklearn.py:235: DeprecationWarning: The seed parameter is deprecated as of version .6.Please use random_state instead.seed is deprecated.
  'seed is deprecated.', DeprecationWarning)
C:\Users\Administrator\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.4, gamma=0,
              learning_rate=0.07, max_delta_step=0, max_depth=3,
              min_child_weight=1.5, missing=None, n_estimators=1000, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0.75, reg_lambda=0.45, scale_pos_weight=1, seed=42,
              silent=None, subsample=0.6, verbosity=1)


In [13]:
actual = y_val
predicted = y_pred
results = confusion_matrix(actual, predicted) 
print('Confusion Matrix :')
print(results) 
print('Accuracy Score :',accuracy_score(actual, predicted))
print('Report : ')
print(classification_report(actual, predicted))

Confusion Matrix :
[[2864    8    0    7    0    0  102    2  261]
 [ 367   11    1   30    0    1   27    0  271]
 [   0    0   17   27    0    0    1    0    2]
 [  55    0    5  735    0    0   59    0  136]
 [ 409    2    0    2    1    0   22    1  231]
 [ 407    1    0    7    0    0   22    0  210]
 [ 231    0    0   33    0    0 2064    0  126]
 [ 349    0    0    5    0    0   27    0  215]
 [ 514    3    0   53    1    2  102    0 1922]]
Accuracy Score : 0.6371014977826124
Report : 
                 precision    recall  f1-score   support

       Adoption       0.55      0.88      0.68      3244
           Died       0.44      0.02      0.03       708
       Disposal       0.74      0.36      0.49        47
     Euthanasia       0.82      0.74      0.78       990
        Missing       0.50      0.00      0.00       668
       Relocate       0.00      0.00      0.00       647
Return to Owner       0.85      0.84      0.85      2454
      Rto-Adopt       0.00      0.00      0.0

In [14]:
from sklearn.ensemble import RandomForestClassifier

# Create the model with 100 trees
model = RandomForestClassifier(n_estimators=100, 
                               bootstrap = True,
                               max_features = 'sqrt')
# Fit on training data
model.fit(X_train,y_train)
y_pred = model.predict(X_val)
print (model)

actual = y_val
predicted = y_pred
results = confusion_matrix(actual, predicted) 
print('Confusion Matrix :')
print(results) 
print('Accuracy Score :',accuracy_score(actual, predicted))
print('Report : ')
print(classification_report(actual, predicted))
#plot_confusion

C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='sqrt', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
Confusion Matrix :
[[2832   13    0    4   16   21  155   19  184]
 [ 380   15    0   22    7    4   38    2  240]
 [   0    1   17   27    0    0    1    0    1]
 [  67    3    5  675    0    0   90    0  150]
 [ 404    6    0    2    7   10   34    9  196]
 [ 396    7    0    9    9    3   41    8  174]
 [ 259    0    0   25    0    0 2070    1   99]
 [ 352   10    0    0    5    5   42    4  178]
 [ 579   44    0   42   46   34  162   40 1650]]
Accuracy Score : 0.608568320642624
Report : 
            

In [25]:
from hyperopt import hp, fmin, tpe
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing

X = all_df[all_df.columns[:-1]]
y = all_df[RESPONSE]
print(X.shape, y.shape)

space = {
    'n_estimators': hp.choice('n_estimators', range(1, 500)),
    'max_depth': hp.choice('max_depth', range(3, 100)),
    'learning_rate': hp.quniform('learning_rate', 0.01, 0.1, 0.01),
    'silent': 1,
    'objective': 'multi:softprob',
    'nthread': -1, # set -1 to use all available scores
    'gamma': hp.quniform('gamma', 0.1, 2, 0.01),
    'min_child_weight': hp.quniform('min_child_weight', 0, 5, 0.5),
    'subsample': hp.quniform('subsample', 0.1, 0.9, 0.05),
    'colsample_bytree': hp.quniform('colsample_bytree', 0.1, 0.9, 0.05),
    'colsample_bylevel': hp.quniform('colsample_bylevel', 0.1, 0.9, 0.05),
    #'colsample_bynode': hp.quniform('colsample_bynode', 0.1, 0.9, 0.05),
    'reg_alpha' : hp.quniform('reg_alpha', 0, 1, 0.00001), # this parameter is for L1 regulation
    'reg_lambda' : hp.quniform('reg_lambda', 0, 20, 0.1)
}

def objective(space):
    X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.25,random_state=0)
    
    le = preprocessing.LabelEncoder()
    y_train = le.fit_transform(y_train)
    y_val = le.transform(y_val)

    model = xgb.XGBClassifier(
            colsample_bylevel=space['colsample_bylevel'],
            colsample_bytree=space['colsample_bytree'],
            #colsample_bynode=space['colsample_bynode'],
            gamma=space['gamma'],
            learning_rate=space['learning_rate'],
            max_depth=space['max_depth'],
            min_child_weight=space['min_child_weight'],
            n_estimators=space['n_estimators'],
            nthread=space['nthread'],
            objective=space['objective'],
            random_state=0,
            reg_alpha=space['reg_alpha'], 
            reg_lambda=space['reg_lambda'],
            scale_pos_weight=1, 
            seed=None,
            silent=space['silent'],
            subsample=space['subsample'],
            verbosity=1
    )
    model.fit(X_train, y_train)
    return mean_squared_error(y_val,model.predict(X_val))



best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=100
)
print(best)

(47802, 629) (47802, 1)
  1%|▍                                             | 1/100 [28:05<46:21:37, 1685.84s/it, best loss: 10.114132708559953]

C:\Users\Administrator\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)

C:\Users\Administrator\Anaconda3\lib\site-packages\xgboost\sklearn.py:242: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)

C:\Users\Administrator\Anaconda3\lib\site-packages\xgboost\sklearn.py:249: DeprecationWarning: The silent parameter is deprecated.Please use verbosity instead.silent is depreated
  'silent is depreated', DeprecationWarning)



  2%|▉                                             | 2/100 [33:19<34:40:59, 1274.08s/it, best loss: 10.114132708559953]

C:\Users\Administrator\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)

C:\Users\Administrator\Anaconda3\lib\site-packages\xgboost\sklearn.py:242: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)

C:\Users\Administrator\Anaconda3\lib\site-packages\xgboost\sklearn.py:249: DeprecationWarning: The silent parameter is deprecated.Please use verbosity instead.silent is depreated
  'silent is depreated', DeprecationWarning)



  3%|█▍                                             | 3/100 [34:04<24:23:35, 905.31s/it, best loss: 10.114132708559953]

C:\Users\Administrator\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)

C:\Users\Administrator\Anaconda3\lib\site-packages\xgboost\sklearn.py:242: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)

C:\Users\Administrator\Anaconda3\lib\site-packages\xgboost\sklearn.py:249: DeprecationWarning: The silent parameter is deprecated.Please use verbosity instead.silent is depreated
  'silent is depreated', DeprecationWarning)



  3%|█▍                                            | 3/100 [57:54<31:12:31, 1158.26s/it, best loss: 10.114132708559953]


KeyboardInterrupt: 

In [9]:
best = {'colsample_bylevel':0.7000000000000001,
        'colsample_bytree':0.8,
        'gamma':.033,
        'learning_rate':0.05,
        'max_depth':68,
        'min_child_weight':4.0,
        'n_estimators':393,
        'reg_alpha':0.45844,
        'reg_lambda':17.7,
        'subsample':0.65    
}

X = all_df[all_df.columns[:-1]]
y = all_df[RESPONSE]
print(X.shape, y.shape)

(47802, 629) (47802, 1)


In [13]:
best_model = xgb.XGBClassifier(
            colsample_bylevel=best['colsample_bylevel'],
            colsample_bytree=best['colsample_bytree'],
            #colsample_bynode=best['colsample_bynode'],
            gamma=best['gamma'],
            learning_rate=best['learning_rate'],
            max_depth=best['max_depth'],
            min_child_weight=best['min_child_weight'],
            n_estimators=best['n_estimators'],
            random_state=0,
            reg_alpha=best['reg_alpha'], 
            reg_lambda=best['reg_lambda'],
            scale_pos_weight=1, 
            seed=None,
            subsample=best['subsample'],
            verbosity=1
)

X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.25,random_state=0)

le = preprocessing.LabelEncoder()
y_train = le.fit_transform(y_train)
y_val = le.transform(y_val)

C:\Users\Administrator\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [14]:
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_val)
actual = y_val
predicted = y_pred
results = confusion_matrix(actual, predicted) 
print('Confusion Matrix :')
print(results) 
print('Accuracy Score :',f1_score(actual, predicted, average='micro'))
print('Report : ')
print(classification_report(actual, predicted))

Confusion Matrix :
[[2842   12    0   10    9   11   92   11  257]
 [ 365   21    0   31    0    1   29    2  259]
 [   0    0   13   31    0    0    1    0    2]
 [  54    2    3  750    3    0   58    0  120]
 [ 393    2    0    5    4    2   25    3  234]
 [ 389    5    0   11    6    2   26    1  207]
 [ 225    1    0   44    0    0 2065    2  117]
 [ 340    1    0    5    5    2   24    1  218]
 [ 438   26    0   54   19   11   99   14 1936]]
Accuracy Score : 0.6387749979081249
Report : 
              precision    recall  f1-score   support

           0       0.56      0.88      0.69      3244
           1       0.30      0.03      0.05       708
           2       0.81      0.28      0.41        47
           3       0.80      0.76      0.78       990
           4       0.09      0.01      0.01       668
           5       0.07      0.00      0.01       647
           6       0.85      0.84      0.85      2454
           7       0.03      0.00      0.00       596
           8   

In [16]:
best_model.save_model('../models/best_xgb.pkl')

### SVM WITHOUT OVERSAMPLING

In [18]:
from sklearn.svm import SVC
svclassifier = SVC(kernel='rbf',
                  gamma=0.01)

X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.25,random_state=0)

le = preprocessing.LabelEncoder()
y_train = le.fit_transform(y_train)
y_val = le.transform(y_val)

svclassifier.fit(X_train, y_train)
y_pred = svclassifier.predict(X_val)
actual = y_val
predicted = y_pred
results = confusion_matrix(actual, predicted) 
print('Confusion Matrix :')
print(results) 
print('Accuracy Score :',f1_score(actual, predicted, average='micro'))
print('Report : ')
print(classification_report(actual, predicted))

KeyboardInterrupt: 